In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [4]:
#Import reference LAI data
lai = pd.read_csv('./data/compiled_data_formodel.csv')
mergedData = pd.DataFrame(columns = 
                        list(lai.columns) + ['UnionMean', 'UnionSD', 'TotalMean', 'TotalSD', 'OverlapMean', 'OverlapSD'])

In [7]:
for index in range(0, 100):
    #Import data one site at a time
    load_path = './data/modeloutput/rhino-data/canopyUnion-site-' + str(index) + '.txt'
    load_path_total = './data/modeloutput/rhino-data/canopyTotal-site-' + str(index) + '.txt'
    dat = pd.read_csv(load_path, header = None, 
                    names = ['Values'])
    datTot = pd.read_csv(load_path_total, header = None, 
                    names = ['Values'])
   
    #Get differences
    dat['delta'] = np.diff(dat['Values'], append = [0])

    #Sort out peaks
    negs = np.where(dat.delta < 0)[0]
    peaks = []
    totpeaks = []
    overlap = []
    #Take only peask where the union volume could reasonably be calculated
    for neg in negs:
        if(dat.Values[min(neg + 1, len(dat.Values) -1)] != -1):
            peaks.append(dat.Values[neg])
            totpeaks.append(datTot.Values[neg])
            overlap.append(datTot.Values[neg] - dat.Values[neg])

    #Write row
    row = lai.iloc[index]
    row['UnionMean'] = np.nanmean(peaks)
    row['UnionSD'] = np.std(peaks)
    row['TotalMean'] = np.nanmean(totpeaks)
    row['TotalSD'] = np.std(totpeaks)
    row['OverlapMean'] = np.nanmean(overlap)
    row['OverlapSD'] = np.std(overlap)

    #Save
    mergedData.loc[index] = row

C:\Users\marie\AppData\Local\Temp\ipykernel_19584\651688236.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['UnionMean'] = np.nanmean(peaks)
C:\Users\marie\AppData\Local\Temp\ipykernel_19584\651688236.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['UnionMean'] = np.nanmean(peaks)
C:\Users\marie\AppData\Local\Temp\ipykernel_19584\651688236.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['UnionSD'] = np.std(pe

In [8]:
#Export Data
mergedData.to_csv('./data/modeloutput/rhino-data/processedData.csv', index = False)